# Network flow optimization model

## Sets & Subsets

* $N$: Set of nodes in the network, i $\in$ {0,1,…,N} 

* $A$: Set of arcs in the network, k $\in$ {0,1,…,A}

* $S$ $\subset N$: Subset of nodes that represent the beginning of the network (source), s $\in$ {0,1,…,S}

* $Z$ $\subset N$: Subset of nodes that represent the end of the network (sink), z $\in$ {0,1,…,Z}

## Parameters

#### Node's parameters

* $l^{n}_{i}$: Is the lower storing capacity for node ${i}$, $\forall i \in N$

* $u^{n}_{i}$: Is the upper storing capacity for node ${i}$, $\forall i \in N$

* $c^{n}_{i}$: Is the unit cost for node ${i}$, $\forall i \in N$

* $\gamma^{n}_{i}$: Is the initial amount in the network for node ${i}$, $\forall i \in S \subset N$


#### Arc's parameters

* $l^{a}_{i,j}$: Is the lower storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $u^{a}_{i,j}$: Is the upper storing capacity for arc ${(i,j)}$, $\forall (i,j) \in A$

* $c^{a}_{i,j}$: Is the unit transportation cost from node ${i}$ to node ${j}$, $\forall (i,j) \in A$

## Decision variables

#### Node's decisions

* $\beta^{n}_{i}$: Is the amount to be stored in the node ${i}$, $\forall i \in N$

#### Arc's decisions

* $\alpha^{a}_{i,j}$: Is the amount to be send from node ${i}$ to node ${j}$, $\forall (i,j) \in A$

## Objective function 
**Maximize total flow and minimize network costs**

$$ \ Min\space \ Z = \sum_{i \in N} c^{n}_{i}*\beta^{n}_{i} + \sum_{(i,j) \in A} c^{a}_{i,j}*\alpha^{a}_{i,j} $$

$$ \ Max\space \ Z = \sum_{(i,j) \in A} \alpha^{a}_{i,j} $$

## Constraints

#### Node's capacity
$$ l^{n}_{i} \leq  \beta^{n}_{i} \leq u^{n}_{i}, \forall i \in N $$

#### Arc's capacity
$$ l^{a}_{i,j} \leq  \alpha^{a}_{i,j} \leq u^{a}_{i,j}, \forall (i,j) \in A $$

#### Flow balancing equations
Flow balancing (except for initial and ending nodes):
$$ \sum_{(i,j) \in A} \alpha^{a}_{i,j} - \sum_{(j,k) \in A} \alpha^{a}_{j,k} = \beta^{n}_{j}, \forall j \in N - S \cup Z  $$
Initial nodes flow balance:
$$ \sum_{(s,j) \in A} \alpha^{a}_{s,j} + \beta^{n}_{s} = \gamma^{n}_{s}, \forall s \in S \subset N$$
Ending nodes flow balance:
$$ \sum_{(i,z) \in A} \alpha^{a}_{i,z} = \beta^{n}_{z}, \forall z \in Z \subset N$$

### Nonnegative constraints
$$ \alpha^{a}_{i,j} \geq 0, \forall (i,j) \in A $$
$$ \beta^{n}_{i} \geq 0, \forall i \in N $$


In [1]:
import numpy as np 
import pandas as pd
import os
import pyomo.environ as pe
from collections import defaultdict
import itertools
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
URL_cf_path = 'https://github.com/joaquinrovi/Optimization/blob/master/conf_file.xlsx?raw=true'

In [3]:
#Read arc's raw data from the configuration file
arcs_raw_data = pd.read_excel(URL_cf_path, sheet_name='Arcs')
#Read node's raw data from the configuration file
initial_nodes_raw_data = pd.read_excel(URL_cf_path, sheet_name='Node_start')
ending_nodes_raw_data = pd.read_excel(URL_cf_path, sheet_name='Node_end')
nodes_raw_data = pd.read_excel(URL_cf_path, sheet_name='Nodes')

In [4]:
columns = ['Node', 'Min_Cap', 'Max_Cap', 'Cost_node', 'Active']
df = [initial_nodes_raw_data[columns],
    ending_nodes_raw_data[columns],
    nodes_raw_data[columns]]
nodes_data = pd.concat(df)

In [5]:
arcs_data = pd.merge(left=arcs_raw_data,\
    right=nodes_data[['Node','Cost_node']],\
    left_on='Node_end',\
    right_on='Node',\
    how='left').drop('Node',1)

C:\Users\JRODRI~1.VIL\AppData\Local\Temp/ipykernel_19428/3900381228.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  arcs_data = pd.merge(left=arcs_raw_data,\


In [6]:
#Cleaning arcs
arcs_data= arcs_data[arcs_data['Active']=="Y"].drop('Active', 1)
arcs_data= arcs_data.drop_duplicates().set_index(['Node_start', 'Node_end'])
#Cleaning nodes
nodes_data = nodes_data[nodes_data['Active']=="Y"].drop('Active',1)
nodes_data = nodes_data.drop_duplicates().set_index('Node')

initial_nodes_data = initial_nodes_raw_data[initial_nodes_raw_data['Active']=="Y"].drop('Active',1)
initial_nodes_data = initial_nodes_data.drop_duplicates().set_index('Node')

ending_nodes_data = ending_nodes_raw_data[ending_nodes_raw_data['Active']=="Y"].drop('Active',1)
ending_nodes_data = ending_nodes_data.drop_duplicates().set_index('Node')

C:\Users\JRODRI~1.VIL\AppData\Local\Temp/ipykernel_19428/2693748716.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  arcs_data= arcs_data[arcs_data['Active']=="Y"].drop('Active', 1)
C:\Users\JRODRI~1.VIL\AppData\Local\Temp/ipykernel_19428/2693748716.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nodes_data = nodes_data[nodes_data['Active']=="Y"].drop('Active',1)
C:\Users\JRODRI~1.VIL\AppData\Local\Temp/ipykernel_19428/2693748716.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  initial_nodes_data = initial_nodes_raw_data[initial_nodes_raw_data['Active']=="Y"].drop('Active',1)
C:\Users\JRODRI~1.VIL\AppData\Local\Temp/ipykernel_19428/2693748716.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame

# Define dictionaries and lists

In [7]:
nodes = set(nodes_data.index)
initial_nodes = set(initial_nodes_data.index)
ending_nodes = set(ending_nodes_data.index)
arcs = set(arcs_data.index)
exits = defaultdict(set)
entry = defaultdict(set)

for (i,j) in set(arcs_data.index):
    exits[i].add(j)
    entry[j].add(i)

# Optimization model
## Sets & subsets

In [9]:
model = pe.ConcreteModel('Network_flow')
#Nodes
model.nodes = pe.Set(initialize = nodes)
model.initial = pe.Set(within = model.nodes, initialize = initial_nodes)
model.ending = pe.Set(within = model.nodes, initialize = ending_nodes)
#Arcs
model.arcs = pe.Set(within = model.nodes * model.nodes, initialize = arcs)
model.entry = pe.Param(model.nodes, initialize = entry, default=set(), within=pe.Any)
model.exit = pe.Param(model.nodes, initialize = exits, default=set(), within=pe.Any)

    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


## Parameters

In [10]:
#Nodes
model.max_capacity = pe.Param(model.nodes, initialize = nodes_data['Max_Cap'].to_dict())
model.min_capacity = pe.Param(model.nodes, initialize = nodes_data['Min_Cap'].to_dict())
model.node_costs = pe.Param(model.nodes, initialize = nodes_data['Cost_node'].to_dict())
model.water_in = pe.Param(model.initial, initialize = initial_nodes_data["Initial"].to_dict())
#Arcs
model.max_flow = pe.Param(model.arcs, initialize = arcs_data["Max_flow"].to_dict())
model.min_flow = pe.Param(model.arcs, initialize = arcs_data["Min_flow"].to_dict())
model.arc_costs = pe.Param(model.arcs, initialize = arcs_data["Cost_arc"].to_dict())

## Decision variables

In [11]:
#Node's decisions
model.beta = pe.Var(model.nodes, domain = pe.NonNegativeReals)
#Arc's decisions
model.alpha = pe.Var(model.arcs, domain = pe.NonNegativeReals)

## Objetive functions (cost and flow)

In [12]:
def min_cost(model):
    node_costs = sum(model.beta[i] * model.node_costs[i] for i in model.nodes)
    arc_costs = sum(model.alpha[i,j] * model.arc_costs[i,j] for (i,j) in model.arcs)
    return node_costs + arc_costs
model.obj_function_cost = pe.Objective(sense = pe.minimize, rule = min_cost)
#model.obj_function_cost.deactivate()
def max_flow(model):
    max_flow = sum(model.alpha[i,j] for (i,j) in model.arcs if j not in list(model.ending))    
    return max_flow   
model.obj_function_flow = pe.Objective(sense = pe.maximize, rule = max_flow)
model.obj_function_flow.deactivate()